In [11]:

path =r"C:\Users\S K Saxena\Desktop\Sem 3\DAI-101\creditcard.csv"

print("Path to dataset files:", path)

Path to dataset files: C:\Users\S K Saxena\Desktop\Sem 3\DAI-101\creditcard.csv


In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.utils.class_weight import compute_class_weight 

In [15]:
data=pd.read_csv(path)
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [17]:
X=data.drop('Class',axis=1)
y=data['Class']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)
            

In [23]:
rus=RandomUnderSampler(random_state=42)
X_rus,y_rus=rus.fit_resample(X_train,y_train)

In [25]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train) 

In [63]:
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train) 
tl = TomekLinks()
X_tl, y_tl = tl.fit_resample(X_train, y_train) 
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights)) 

In [65]:
def evaluate_model(X_train, y_train, X_test, y_test,method_name, class_weights=None):
    model = LogisticRegression(class_weight=class_weights, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    report = classification_report(y_test, y_pred, output_dict=True)  # output_dict for structured data
    auc = roc_auc_score(y_test, y_prob)
    
   
    results = {
        "Method": method_name,
        "Precision": report['1']['precision'],  # Assuming class "1" is the minority
        "Recall": report['1']['recall'],
        "F1-Score": report['1']['f1-score'],
        "AUC": auc
    }
    return results

In [66]:
results_list = []

results_list.append(evaluate_model(X_rus, y_rus, X_test, y_test, method_name="Random Undersampling"))
results_list.append(evaluate_model(X_ros, y_ros, X_test, y_test, method_name="Random Oversampling"))
results_list.append(evaluate_model(X_smote, y_smote, X_test, y_test, method_name="SMOTE"))
results_list.append(evaluate_model(X_tl, y_tl, X_test, y_test, method_name="Tomek Links"))
results_list.append(evaluate_model(X_train, y_train, X_test, y_test, method_name="Class Weights", class_weights=class_weights_dict))


results_df = pd.DataFrame(results_list)
print(results_df)

sorted_results = results_df.sort_values(by="AUC", ascending=False)
print("\nSorted Results by AUC:\n", sorted_results)



D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

                 Method  Precision    Recall  F1-Score       AUC
0  Random Undersampling   0.037557  0.928571  0.072194  0.971889
1   Random Oversampling   0.043838  0.918367  0.083682  0.977451
2                 SMOTE   0.084795  0.887755  0.154804  0.977394
3           Tomek Links   0.684932  0.510204  0.584795  0.849454
4         Class Weights   0.047273  0.928571  0.089965  0.978944

Sorted Results by AUC:
                  Method  Precision    Recall  F1-Score       AUC
4         Class Weights   0.047273  0.928571  0.089965  0.978944
1   Random Oversampling   0.043838  0.918367  0.083682  0.977451
2                 SMOTE   0.084795  0.887755  0.154804  0.977394
0  Random Undersampling   0.037557  0.928571  0.072194  0.971889
3           Tomek Links   0.684932  0.510204  0.584795  0.849454


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
